In [1]:
import numpy as np
import os
import sys

module_path = os.path.abspath(os.path.join("../lib"))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pk_tools import pk_tools
import pk_fit_selection as fit

In [2]:
mocks_path = "../data/BOSS_DR12_NGC_z1/Patchy_V6C_BOSS_DR12_NGC_z1/"
all_filenames = [f for f in os.listdir(mocks_path) if not f.startswith('.')]
np.random.shuffle(all_filenames)

# Bin selection
kmin = 0.01 # h/Mpc
kmax = 0.1 # h/Mpc
poles = [True, False, True, False, False]
p = 18 # The above correspond to two multipoles of 9 bins each

# We create 90 sets of each 21 mocks and 63 sets of each 30 mocks
total = 1890 # = 21*30*3
filenames = all_filenames[:total] # Select #total mocks from all available mocks (=2048)

In [3]:
def create_mocks_matrices(n, filenames):
    save_path = f"../output/BOSS_DR12_NGC_z1/mocks/n{n}/"
    r = len(filenames) // n
    
    for v in range(r):
        filenames_cut = filenames[v*n:(v+1)*n]
        P = np.empty((p, len(filenames_cut)))
    
        for k, filename in enumerate(filenames_cut):
            
            # Load the data using Beutler's pk_tools module 
            data = pk_tools.read_power(os.path.join(mocks_path, filename), combine_bins=10)
            kbins, pk_data_vector = pk_tools.dict_to_vec(data)
        
            fit_selection = fit.get_fit_selection(kbins, kmin=kmin, kmax=kmax, pole_selection=poles)
            fit_pk_data_vector = pk_data_vector[fit_selection]
        
            P[:,k] = fit_pk_data_vector
    
        np.savetxt(os.path.join(save_path, f"P_{p}_{n}_v{v+1}.matrix"), P)

In [4]:
# 90 sets of each 21 mocks
create_mocks_matrices(21, filenames)

# Shuffle for next batch
np.random.shuffle(filenames)

# 63 sets of each 30 mocks
create_mocks_matrices(30, filenames)

# All mocks
create_mocks_matrices(2048, all_filenames)